<a href="https://colab.research.google.com/github/niiroo/Gemini_Colab/blob/main/templates/aistudio_gemini_prompt_freeform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-flash' # @param {isTemplate: true}
contents_b64 = 'W10=' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

In [9]:
import os
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get("GOOGLE_API_KEY")

genai.configure(api_key=api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
      {
      "role": "user",
      "parts": [
        "あなたは感情を深掘りする質問を得意とするアシスタントです。ユーザーが書いた文章に基づいて、そのときの感情や体験の背景を詳しく引き出し、価値観、強みを見つけ出すことにつながるような質問を1つしてください。質問は親しみやすく、ユーザーがリラックスして答えやすいトーンで作成してください。\n",
      ],
    },
    {
      "role": "model",
      "parts": [
          "はい、私は感情を深掘りする質問を得意とするアシスタントです。リラックスしたトーンで、ユーザーが答えやすい簡単な質問を作成します。"
      ]
    }
  ]
)

response = chat_session.send_message("今日はパンケーキを食べた")

print(response.text)

ふむふむ、今日はパンケーキを食べたんですね！  それはどんな気持ちでしたか？  パンケーキを食べることで、どんな気持ちになったり、思い出したりしましたか？  （例えば、子どもの頃によく食べたとか、誰かと一緒に食べたとか、特別な日だったとか…どんな些細なことでも構いませんよ！）



In [11]:
response = chat_session.send_message("おいしくて嬉しい気持ちになった")

print(response.text)

「おいしくて嬉しい気持ちになった」とのこと、ありがとうございます。  それは素敵な気持ちですね！  もう少し深堀りさせてください。  その「嬉しい」気持ちは、どんな種類の嬉しさでしたか？  例えば、

* **単純な満足感**：美味しいものを食べて、五感が満たされたシンプルな喜び？
* **幸福感**：特別な時間や、大切な人と過ごした思い出とリンクした喜び？
* **達成感**：自分で作ったパンケーキなら、作った達成感と相まっての喜び？
* **安心感**：日常のちょっとした幸せを感じた安心感？

どれに一番近いですか？  もしくは、他に言葉で表現できるものがあれば教えてください。  どんな小さなニュアンスでも構いません。  そこから、あなたの価値観や強みが見えてくるかもしれませんよ。



In [12]:
response = chat_session.send_message("幸せな気持ちになりました！友人と一緒に食べたこととパンケーキのふわふわ感のおかげです")

print(response.text)

友人と一緒に食べたことと、パンケーキのふわふわ感が幸せな気持ちに繋がったんですね！素晴らしいですね。  では、もう少し具体的に教えていただけますか？

友人と食べることで、どんな幸せを感じましたか？  例えば、一緒にいることの楽しさ、会話の弾み、共有した時間への満足感など、どんな些細な事でも構いません。  そして、ふわふわのパンケーキの食感は、あなたにとってどんな意味を持っていましたか？  例えば、子供の頃を思い出したり、安心感を感じたり、特別な日の記憶と繋がったり…  これらの幸せな感情から、あなたの大切にしていることや、あなたがどんな人なのかが垣間見えますよ。  教えていただけると嬉しいです！



In [14]:
response = chat_session.send_message("これまでのやり取りを基に、日記として200字程度で要約してまとめてください。")
print(response.text)

今日は友人とパンケーキを食べた。ふわふわの食感と甘さが口いっぱいに広がり、幸せな気持ちになった。友人とのおしゃべりも楽しかった。単純な出来事だけど、この幸せな時間は、日々の忙しさの中でとても貴重だった。  美味しいものを食べる喜び、そして大切な人と過ごす時間、この二つが私にとっての幸せの源泉だと改めて感じた。  ふわふわのパンケーキは、まるで幼い頃の優しい記憶を呼び起こすようだった。  この幸せな気持ちを忘れずに、明日からも頑張ろうと思える。  小さな幸せを見つける感性と、それを共有できる友人との絆を大切にしたい。  今日のパンケーキは、単なる食事ではなく、心の栄養になった。



In [13]:
response = chat_session.send_message("これまでのやり取りを基に、以下の点を要約してください:\n1. 主な出来事や体験\n2. 感情の変化や価値観\n3. ユーザーの強みや特徴\n簡潔かつ自己分析に役立つ形でまとめてください。")

print(response.text)

**1. 主な出来事や体験:** 友だちとふわふわのパンケーキを食べた。

**2. 感情の変化や価値観:**  美味しいパンケーキと友人との時間を共有することで、大きな幸福感を感じた。  この体験から、人間関係と五感を通して感じる喜びを大切にする価値観が伺える。

**3. ユーザーの強みや特徴:**  些細な幸せをしっかりと感じ取り、喜びを分かち合える社交性と、五感に敏感な繊細さを持っている。  また、幸福感を言語化し、自身の感情を客観的に分析しようとする洞察力も持ち合わせている。



## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
# @title Show the conversation, in colab.
import mimetypes

def show_file(file_data):
    mime_type = file_data["mime_type"]

    if drive_id := file_data.get("drive_id", None):
        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        name = path
        # data = path.read_bytes()
        kwargs = {"filename": path}
    elif url := file_data.get("url", None):
        name = url
        kwargs = {"url": url}
        # response = requests.get(url)
        # data = response.content
    elif data := file_data.get("inline_data", None):
        name = None
        kwargs = {"data": data}
    elif name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files to "
                'Colab using the file manager ("📁 Files"in the left toolbar)'
            )
    else:
        raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")

        print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")
        return

    format = mimetypes.guess_extension(mime_type).strip(".")
    if mime_type.startswith("image/"):
        image = IPython.display.Image(**kwargs, width=256)
        IPython.display.display(image)
        print()
        return

    if mime_type.startswith("audio/"):
        if len(data) < 2**12:
            audio = IPython.display.Audio(**kwargs)
            IPython.display.display(audio)
            print()
            return

    if mime_type.startswith("video/"):
        if len(data) < 2**12:
            audio = IPython.display.Video(**kwargs, mimetype=mime_type)
            IPython.display.display(audio)
            print()
            return

    print(f"File:\n    name: {name}\n    mime_type: {mime_type}\n")


for content in gais_contents:
    if role := content.get("role", None):
        print("Role:", role, "\n")

    for n, part in enumerate(content["parts"]):
        if text := part.get("text", None):
            print(text, "\n")

        elif file_data := part.get("file_data", None):
            show_file(file_data)

    print("-" * 80, "\n")